<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Torch_Notebooks/NER_Task_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tunning pre-trained language model for NER (Named-Entity-Recognition)

We will be using Huggingface pretrained T5 model for language translation task. The model is already pretained for on this task but we will fine tune it.

In [1]:
pip install -q pytorch_lightning datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fssp

In [2]:
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from datasets import load_dataset
import evaluate

import dataset from kaggle

In [3]:
data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
label_names = data["train"].features["ner_tags"].feature.names

In [6]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [7]:
data["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [8]:
line1 = ""
line2 = ""
for word,label in zip(data["train"][0]["tokens"],data["train"][0]["ner_tags"]):
  line1 += word + " "
  line2 += label_names[label] + " "
print(line1)
print(line2)

EU rejects German call to boycott British lamb . 
B-ORG O B-MISC O O O B-MISC O O 


In [9]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
tokenizer.is_fast

True

Pass is_split_into_words=True as out sentences are already split into words.

In [11]:
inputs = tokenizer(data["train"][0]["tokens"],is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


Check the words_ids() function of fast tokenizer to know which words are split into subwords.

In [12]:
print(inputs.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


Check the associated labels with sentences.

In [13]:
labels = data["train"][0]["ner_tags"]
alabel = [label_names[l] for l in labels]
print(alabel)

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [14]:
fake_label = labels.copy()
fake_label = [label_names[l] for l in fake_label]
print(fake_label)

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [15]:
acc = evaluate.load("seqeval")

In [16]:
acc.add(predictions=[fake_label],references=[alabel])

In [17]:
acc.compute()["overall_accuracy"]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


1.0

Here, at this point we need to understand that out tokenizer added some special tokens [CLS] and [SEP] and also some words are split into subwords. This introduces a mismatch between out inputs and labels. We need to exapnd out label list to match with our input list. We will replace the special tokens with -100 because this is an index which will be ignored by our loss function (cross entropy). Also 0 for words which are split into subwords. Below function will do that -->

In [18]:
def align_new_labels(word_ids,labels):
  new_labels = []
  for word_id in word_ids:
    if word_id == None:
      new_labels.append(-100)
    else:
      new_labels.append(labels[word_id])
  return new_labels

In [19]:
print(align_new_labels(inputs.word_ids(),labels))

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [20]:
align_new_labels(inputs.word_ids(),labels)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [21]:
def collate_fn(batch):
  tokens = tokenizer([x["tokens"] for x in batch],is_split_into_words=True,padding=True,truncation=True,return_tensors="pt")
  labels = [x["ner_tags"] for x in batch]
  new_labels = []
  for i,label in enumerate(labels):
    word_ids = tokens.word_ids(batch_index=i)
    new_labels.append(align_new_labels(word_ids,label))
  tokens["labels"] = torch.tensor(new_labels)
  return tokens

In [22]:
train = data["train"]
val = data["validation"]

In [23]:
train_df = torch.utils.data.DataLoader(train,batch_size=32,shuffle=True,collate_fn=collate_fn)
val_df = torch.utils.data.DataLoader(val,batch_size=32,collate_fn=collate_fn)

In [24]:
for x in train_df:
  print(x["input_ids"].shape)
  print(x["labels"].shape)
  break

torch.Size([32, 39])
torch.Size([32, 39])


In [25]:
for x in train:
  print(x["tokens"])
  break

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']


In [26]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

Initialise the model with number of labels we have in our database.

In [27]:
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(label_names))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create a lightning module and set low learning rate of 0.0001. We will use the built-in loss function of model.

In [28]:
def convert_to_original(ol):
  prev = ""
  new = []
  for x in ol:
    if x == "O":
      new.append(x)
    elif x == prev and x == "O":
      continue
    elif x != prev:
      new.append(x)
    prev = x
  new.pop(0)
  new.pop(-1)
  return new

In [76]:
class MyModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = model

  def forward(self,input_ids,attention_mask):
    return self.model(input_ids=input_ids,attention_mask=attention_mask)

  def training_step(self,batch,batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    outputs = self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    loss = outputs.loss
    self.log("train_loss",loss,prog_bar=True,logger=True)
    return loss

  def validation_step(self,batch,batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    outputs = self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    loss = outputs.loss
    self.log("val_loss",loss,prog_bar=True,logger=True)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.0001)

In [77]:
pl_model = MyModel()

In [88]:
trainer = pl.Trainer(max_epochs=1)
trainer.fit(pl_model,train_df,val_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | BertForTokenClassification | 107 M  | eval
------------------------------------------------------------
107 M     Trainable params
0         Non-trainable params
107 M     Total params
430.906   Total estimated model params size (MB)
0         Modules in train mode
228       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [79]:
test = data["test"]

Check out the prediction, it has a flaw which needs to be corrected.

In [89]:
for i,x in enumerate(test):
  if i == 3:
    break
  print(f"Original text : {x['tokens']}")
  al = [label_names[i] for i in x["ner_tags"]]
  print(f"Original NER output : {al}")
  tok = tokenizer(x["tokens"],is_split_into_words=True,return_tensors="pt")
  out = pl_model(tok["input_ids"],tok["attention_mask"])
  print(f"Tokenized input : {tok.tokens()}")
  print(f"Token word ids : {tok.word_ids()}")
  probs = torch.nn.functional.softmax(out.logits,dim=-1)[0]
  predictions = torch.argmax(probs,dim=-1)
  ol = [label_names[i] for i in predictions.tolist()]
  print(f"Actual Prediction Ouput : {ol}")
  print("\n\n")

Original text : ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
Original NER output : ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O']
Tokenized input : ['[CLS]', 'S', '##OC', '##CE', '##R', '-', 'J', '##AP', '##AN', 'GE', '##T', 'L', '##UC', '##K', '##Y', 'W', '##IN', ',', 'CH', '##IN', '##A', 'IN', 'S', '##UR', '##PR', '##IS', '##E', 'DE', '##F', '##EA', '##T', '.', '[SEP]']
Token word ids : [None, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 6, 7, 7, 7, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 11, None]
Actual Prediction Ouput : ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'B-MISC', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'B-ORG', 'B-ORG', 'O', 'O', 'O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O']



Original text : ['Nadim', 'Ladki']
Original NER output : ['B-PER', 'I-PER']
Tokenized input : ['[CLS]', 'Na', '##di', '##m', 'La', '##d', '##ki', '[SEP]']
Token word ids : [None, 0, 0, 0, 1, 1, 1,

***Considering second example Actual Prediction Output, we see that sentence has only two words. But looking at our Token word ids our tokeniser has split the words into subwords as can been seen under Tokenized input. This is how we have modified our labels using align_new_labels function and now our model is ouputting in the same way.
To overcome this we need to write a function.
Define a functions which will smooth out prediction to match with original NER output.
Below function will take the output from model after softmax and argmax and converting them from intergers to actual NER tokens. It will take the list and remove the repeated NER tokens ('B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC') which are consecutive but will keep all zeros (0). ***

In [90]:
for i,x in enumerate(test):
  if i == 10:
    break
  print(x["tokens"])
  al = [label_names[i] for i in x["ner_tags"]]
  print(al)
  tok = tokenizer(x["tokens"],is_split_into_words=True,return_tensors="pt")
  out = pl_model(tok["input_ids"],tok["attention_mask"])
  print(tok.word_ids())
  probs = torch.nn.functional.softmax(out.logits,dim=-1)[0]
  predictions = torch.argmax(probs,dim=-1)
  ol = [label_names[i] for i in predictions.tolist()]
  ol = convert_to_original(ol)
  # a = acc.compute(predictions=[ol],references=[al])
  print(ol)
  # print(a)
  print("\n\n")


['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O']
[None, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 6, 7, 7, 7, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 11, None]
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O']



['Nadim', 'Ladki']
['B-PER', 'I-PER']
[None, 0, 0, 0, 1, 1, 1, None]
['I-PER']



['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06']
['B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O']
[None, 0, 0, 0, 0, 1, 2, 3, 4, 5, 5, 5, 5, 5, None]
['B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O']



['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.']
['B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O'

In [91]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [93]:
tt = "Anand works at Muthoot Finance and he lives in Pune."
tok = tokenizer(tt,is_split_into_words=False,return_tensors="pt")
out = pl_model(tok["input_ids"],tok["attention_mask"])
probs = torch.nn.functional.softmax(out.logits,dim=-1)[0]
predictions2 = torch.argmax(probs,dim=-1)
ol = [label_names[i] for i in predictions2.tolist()]
ol = convert_to_original(ol)
print(tt)
print(tok.tokens())
print(ol)

Anand works at Muthoot Finance and he lives in Pune.
['[CLS]', 'Anand', 'works', 'at', 'Mu', '##th', '##oot', 'Finance', 'and', 'he', 'lives', 'in', 'Pune', '.', '[SEP]']
['B-PER', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'B-LOC', 'O']
